In [72]:
from pypdf import PdfReader, PdfWriter
from pypdf.generic import NameObject
import fitz  # PyMuPDF

In [84]:
from pypdf import PdfReader

reader = PdfReader("tester.pdf")
page = reader.pages[0]
print(page.extract_text())

# extract only text oriented up
print(page.extract_text(0))

# extract text oriented up and turned left
print(page.extract_text((0, 90)))

# extract text in a fixed width format that closely adheres to the rendered
# layout in the source pdf
print(page.extract_text(extraction_mode="layout"))

# extract text preserving horizontal positioning without excess vertical
# whitespace (removes blank and "whitespace only" lines)
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

# adjust horizontal spacing
print(page.extract_text(extraction_mode="layout", layout_mode_scale_weight=1.0))

# exclude (default) or include (as shown below) text rotated w.r.t. the page
print(page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False))

Name "                                          "
Passport Number"                                          "
Name "                                          "
Passport Number"                                          "
Name "                                          "
Passport Number"                                          "
Name                             "                                          "
Passport Number                  "                                          "
Name                             "                                          "
Passport Number                  "                                          "
Name                      "                                          "
Passport Number           "                                          "
Name                             "                                          "
Passport Number                  "                                          "


In [56]:
# Load the PDF document
reader = PdfReader("form.pdf")

# Retrieve the form fields; this returns a dictionary where keys are field names.
fields = reader.get_fields()

# Check if any fields were found
if fields:
    print("Form fields found in the document:")
    for field_name, field_info in fields.items():
        print(f"Field Name: {field_name}")
        print("Field Info:")
        for key, value in field_info.items():
            print(f"  {key}: {value}")
        print("-" * 40)
else:
    print("No form fields found in the document.")


Form fields found in the document:
Field Name: PASAPORTE
Field Info:
  /T: PASAPORTE
  /FT: /Tx
  /TU: PASAPORTE
----------------------------------------
Field Name: NIE
Field Info:
  /T: NIE
  /FT: /Tx
  /TU: N.I.E
----------------------------------------
Field Name: nie_principal
Field Info:
  /T: nie_principal
  /FT: /Tx
  /TU: Causa baja por cese
----------------------------------------
Field Name: NIE_2
Field Info:
  /T: NIE_2
  /FT: /Tx
  /TU: N.I.E
----------------------------------------
Field Name: 1er Apellido
Field Info:
  /T: 1er Apellido
  /FT: /Tx
  /TU: 1er Apellido
----------------------------------------
Field Name: 2 Apellido
Field Info:
  /T: 2 Apellido
  /FT: /Tx
  /TU: 1er Apellido
----------------------------------------
Field Name: Nombre
Field Info:
  /T: Nombre
  /FT: /Tx
  /TU: Nombre
----------------------------------------
Field Name: Sexo
Field Info:
  /T: Sexo
  /FT: /Btn
  /Ff: 49152
  /Kids: [IndirectObject(774, 0, 2378607433712), IndirectObject(775, 0, 

In [83]:
from pypdf import PdfReader, PdfWriter
from pypdf.generic import NameObject

# Load the PDF
reader = PdfReader("form.pdf")
writer = PdfWriter()

# Copy all pages
for page in reader.pages:
    writer.add_page(page)

# Copy AcroForm dictionary
if "/AcroForm" in reader.trailer["/Root"]:
    writer._root_object.update({
        NameObject("/AcroForm"): reader.trailer["/Root"]["/AcroForm"]
    })

# DEBUG: Print all available form field names
fields = reader.get_fields()
print("Available Fields:")
for field_name in fields.keys():
    print(f"- {field_name}")

# FIELDS
PASSPORT = "A3932179"
NAME = ["Sebastian", "Perilla", "Espinosa"]
NIE = "Y8865712P"
GENDER = "/Hombre"  # Make sure this matches the available options

# Step 3: Update multiple field values correctly
writer.update_page_form_field_values(
    writer.pages[0],
    {
        "PASAPORTE": PASSPORT,
        "Nombre": " ".join(NAME),
        "NIE": NIE,
        "1er Apellido": NAME[1],
        "2 Apellido": NAME[2],  # Might be "2º Apellido" instead of "2 Apellido"
        "Sexo": GENDER,  # Ensure this matches available states
    }
)

# Set Need Appearances
writer.set_need_appearances_writer()

# Save the updated PDF
with open("modified.pdf", "wb") as output_stream:
    writer.write(output_stream)

print("Modified PDF saved as 'modified.pdf'.")

# Open the modified PDF that you've already updated.
doc = fitz.open("modified.pdf")

# Iterate over pages and overlay the field text onto the page.
for page in doc:
    # Get a list of widgets (form fields) on the page.
    widgets = page.widgets()
    if widgets:
        for widget in widgets:
            # If the widget has a value, overlay it on the page.
            if widget.field_value is not None:
                # Get the rectangle for the field.
                rect = widget.rect
                # Insert the text into the page content.
                # Adjust the position, font size, and color as needed.
                page.insert_text(
                    (rect.x0 + 50, rect.y0 + 15),  # A small offset may help visually.
                    widget.field_value,
                    fontsize=12,
                    color=(0, 0, 0)
                )
# Save the flattened PDF.
doc.save("flattened.pdf")
print("Flattened PDF saved as 'flattened.pdf'.")



Available Fields:
- PASAPORTE
- NIE
- nie_principal
- NIE_2
- 1er Apellido
- 2 Apellido
- Nombre
- Sexo
- Dia_Nacimiento
- Mes_Nacimiento
- Año_Nacimiento
- Lugar
- Pais
- Nacionalidad
- Estado Civil
- Nombre del padre
- Nombre de la madre
- Domicilio en España
- Numero
- Piso_extranjero
- Localidad
- CP
- Teléfono móvil
- email
- Representante legal en su caso
- DNINIEPAS
- titulo
- NombreRazón Social
- DNINIEPAS_representante
- Domicilio en España_2
- N
- Piso
- Localidad_2
- CP_2
- N Piso_representate
- Teléfono móvil_2
- email_representate
- Representante legal en su caso_2
- DNINIEPAS_2
- titulo_representate
- NombreRazón Social_2
- DNINIEPAS_Notificacio
- Domicilio en España_3
- Numero_notificacion
- N Piso
- Localidad_3
- CP_3
- Provincia
- Teléfono móvil_3
- email_notificacion
- CONSIENTO que las comunicaciones y notificaciones se realicen por medios electrónicos 6
- Nombre y apellidos del titular
- Incicial
- Estancia por estudios
- Autorización de estancia por estudios superi

In [ ]:
# Open the modified PDF that you've already updated.
doc = fitz.open("modified.pdf")

# Iterate over pages and overlay the field text onto the page.
for page in doc:
    # Get a list of widgets (form fields) on the page.
    widgets = page.widgets()
    if widgets:
        for widget in widgets:
            # If the widget has a value, overlay it on the page.
            if widget.field_value is not None:
                # Get the rectangle for the field.
                rect = widget.rect
                # Insert the text into the page content.
                # Adjust the position, font size, and color as needed.
                page.insert_text(
                    (rect.x0 + 50, rect.y0 + 15),  # A small offset may help visually.
                    widget.field_value,
                    fontsize=12,
                    color=(0, 0, 0)
                )
# Save the flattened PDF.
doc.save("flattened.pdf")
print("Flattened PDF saved as 'flattened.pdf'.")


MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

MuPDF error: argument error: cannot create appearance stream for  widgets

Flattened PDF saved as 'flattened.pdf'.
